# Prepare Antwerp LoRaWAN data

In [24]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import json

# Data source
The data can be downloaded from [zenodo](https://zenodo.org/record/3904158#.Y9ZAptJBwUE).

- Make "make_data()" functions.
- Clean the raw data, one and for all.

In [84]:
# Use the json-file to load
path = '../../data/antwerp/lorawan_antwerp_2019_dataset.json.txt'

uplinks = pd.read_json(path)

uplinks = uplinks.drop(columns=['payload', 'counter', 'adr', 'airtime', 'dev_eui'])

uplinks = uplinks.rename(columns={'longitude':'lon_tx',
                                 'latitude': 'lat_tx',
                                 'dev_addr': 'dev_id'})

uplinks = uplinks.rename_axis('uplink_id').reset_index()

uplinks['ngws'] = uplinks['gateways'].str.len()

print(f"Loaded uplink data of shape {uplinks.shape}.")

Loaded uplink data of shape (130430, 9).


In [91]:
# Expand json column to link-level data
links_good = uplinks.explode('gateways')

links_good = links_good.reset_index(drop=True)

json_cols = pd.json_normalize(links_good['gateways'])

json_cols = json_cols.drop(columns=['rx_time.ts_type'])

json_cols = json_cols.rename(columns={'rx_time.time':'time', 'id':'gw_id'})

links_good = pd.concat((links_good, json_cols), axis=1)

print(f"Derived link-level (successes only) dataframe of size {links_good.shape}.")

Derived link-level (successes only) dataframe of size (360253, 14).


In [96]:
total_links = links_good.gw_id.nunique() * links_good.uplink_id.nunique()

print(f"There are {total_links:,} links in total.")
print(f"There are {len(links_good):,} successful links.")

There are 5,738,920 links in total.
There are 360,253 successful links.


In [92]:
# (uplink_id, gw_id) tuple becomes the new index for links

# Get all pairs, then match.

# Then attach gateway-level data....

# Attach uplink-level data...

(360253, 10)

In [26]:
# load gateway data
gws_path = '/Volumes/Transcend/lora/data/traffic/lorawan_antwerp_gateway_locations.json'
gws = pd.read_json(gws_path).transpose()

print(f"Loaded gateway df of shape {gws.shape}.")

Loaded gateway df of shape (249, 2).


In [ ]:
# load data
path = '/Volumes/Transcend/lora/data/traffic/antwerp_links_small.csv'
data = pd.read_csv(path)
dfl = pd.DataFrame(data)

size = dfl.shape[0]
print("Dataframe loaded. There are %d rows." % size)

# More recent work

In [6]:
# Shortcuts to input data files
links_geo_file = "../../data/antwerp_geodata.csv"
links_file = "/Users/sander/Desktop/lora-tools/antwerp_links.geojson"
raster_file = "/Volumes/Transcend/lora/data/antwerp-gis/dsm-1m-clipped.tif"

In [23]:
# Load links
links = gpd.read_file(links_file)

links['success'] = links['rssi'].notnull()

print(f"Loaded GeoDataFrame of shape {links.shape}.")

Loaded GeoDataFrame of shape (57464, 29).


In [10]:
cols = [
    'hdop',
    'sf',
    'channel',
    'log_distance'
    
]

# Note: No antenna alt, receiver alt... Would be useful.

Index(['packet_id', 'adr', 'hdop', 'counter', 'lon_tr', 'sf', 'airtime',
       'dev_addr', 'lat_tr', 'dev_eui', 'channel', 'nr_gateways', 'gateway_id',
       'rssi', 'snr', 'rx_time', 'esp', 'x_gw', 'y_gw', 'lat_gw', 'lon_gw',
       'x_tr', 'y_tr', 'distance', 'log_distance', 'ele_tr', 'ele_gw',
       'geometry'],
      dtype='object')

In [22]:
links.groupby('success')['packet_id'].count().plot(kind='bar')

KeyError: 'success'